# 하고 있는 일

## 왜 이런 짓을 하고 있는가
- 원래 우리가 하던 것은 1 ~ 100 사이의 회귀모델을 구축하는 것
- 그런데 1 ~ 100 이라는 값은 '순위'로, '이산'적인 값이다
- 그렇다면 1 ~ 100 사이의 순위값을 구분하는 '분류'모델로 바꿔보는 건 어떨까?
- 그런데 분류 라벨이 1 ~ 100으로 100가지나 있으면 모델이 너무 복잡해질 것 같다(출력층이 100개나 된다!)
- 그러니 순위를 10개씩 적당히 끊어서 top10, top20, ... top100으로 10개로 카테고리화한 뒤 10가지 레이블로 분류를 해보자

## 추가로 무엇이 불만이었는가
- 모델에 입력되는 Feature는 11개가 있다.
- 그 중 Date에 의문을 가지게 되었다.
- 현재 Date는 과거부터 현재까지의 특정 시점만을 나타낼 뿐인 것으로 보인다.
- 즉, Date가 0에 가까울수록 과거의 시점, 최대치(얼만지는 모르겠지만 뭐 800이나 1000쯤 된다)에 가까울수록 현재 시점에 가깝다는 의미이다.
- 다시 말해, Date Feature가 나타내는 것은 현재나 미래가 아닌 과거의 특정 시점에 불과하다.
- 그런데 이것이 우리가 새로운 데이터를 예측하고자 할 때 무슨 의미가 있는가?
- 우리가 궁극적으로 하고자 하는 것은 새로운 곡을 만든 후 이 곡의 빌보드 순위를 예측하는 것이다.
- 그런데 우리가 과거로 가서 곡을 발매할 것도 아니고 입력값에 과거 시점을 넣는 것이 의미가 있는가?

## 데이터에 이상값도 발견하였다.
- 데이터를 유심히 살펴보던 중 Mode라는 Feature에 주목했다.
- Mode는 문서에 따르면 곡의 키를 의미하는 것으로, 1이 메이저 키, 0이 마이너 키라고 적혀있다. 
- 출처는 여기 : https://developer.spotify.com/documentation/web-api/reference/get-audio-features
- 그런데 데이터의 일부 행의 Mode는 2로 설정되어 있었다.
- 0이 단조고 1이 장조니까 2는 울트라슈퍼장조인가? 뭔가 이상하다.
- 그래서 Mode가 2인 행들을 확인해보니 해당 데이터들은 모든 Feature가 최대치로 설정되어 있는 등 정상적으로 보이지 않는 값이 설정되어 있었다.
- 따라서 Mode가 2인 데이터들은 결측치에 해당하는 데이터로, 정상적인 값이 아니라고 추정되었다.
- 실제로 Mode가 2인 데이터들을 제거해보니 다른 Feature들의 최대값이 1씩 줄어드는 현상이 발생했다.

## 그래서 무엇을 했는가


### 데이터 추가 전처리

#### Date값 처리
- 우선 위에서 언급한 Date와 이상값 문제로 인해 기존 데이터에 추가적으로 전처리를 진행하였다.
- 'Date' Feature을 발매한 월(1월~12월)로 대체하고 해당 Feature 이름을 'Month'로 변경하였다.

#### 이상치(결측치) 처리
- Mode = 2인 값은 이상치 또는 결측치로 간주하고 제거하였다.
  
#### 데이터 순서 무작위화
- 또한 원본 데이터의 순서는 1~100까지 순위가 매주 반복되는 값이다.
- 따라서 정확한 예측을 위해 데이터의 순서를 섞어서 순위가 무작위로 나오게 조정하였다.
  
#### 레이블값 카테고리화
- 레이블값도 1~100 사이의 100가지 값이 저장되어 있었는데 10~100 사이의 10가지 값을 저장하도록 카테고리화하였다.

 > 위 모든 작업을 하는 것이 classify_modeling_sample.ipynb 파일에 있는  df_preprocessing() 함수이다.

### 분류 모델 선택
- 분류 모델은 크게 의사결정나무, SVM(서포트 벡터 머신), KNN(K-최대 근접 이웃) 3가지로 구분하였다.
- SVM은 학습 시간이 무려 163분이 걸린 데다가 성능도 나머지에 비해 눈에 띄게 좋지 않았다.
- 따라서 SVM은 해고시키고 의사결정나무와 KNN의 결과를 살펴보았다.
- 성능이 비슷비슷하여 둘 다 튜닝을 적용해보기로 하였다.

### 하이퍼파라미터 튜닝
- modeling.ipynb에 KNN GridSearch를 위한 코드가 친절하게 적혀있어서 그걸 갖다 쓰기로 했다.
- KNN의 경우 원래 회귀 모델을 위한 코드였지만 공식 문서를 보니 분류 모델에도 똑같은 하이퍼 파라미터를 사용한다고 적혀있다.
- 출처 : https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
- 해당 코드를 적당히 입맛에 맞게 개조한 후 최적 파라미터를 찾아서 적용해보았다.

> 위 하이퍼파라미터 튜닝은 knn_classifier_tuning.ipynb, decision_tree_classifier_tuning.ipynb에서 각각 진행했다.

### 결과
- 튜닝한 의사결정나무와 KNN을 비교해본 결과 2023년 데이터를 기준으로 봤을 때 의사결정나무나 좀 더 결과가 좋아 보인다.